# Setup

In [1]:
import os
import math
import sys
import torch
import re
import numpy as np
from types import SimpleNamespace as ns
from collections import namedtuple

In [2]:
dim3 = namedtuple("dim3", ["x", "y", "z"], defaults=(1, 1))

In [3]:
d = dim3(2, 3)
d

dim3(x=2, y=3, z=1)

In [4]:
d.x, d.y

(2, 3)

In [5]:
np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

In [6]:
sys.path.insert(0, "..")
from utils import show_img, load_cuda, cuda_begin, cdiv

In [7]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.manual_seed(42);

In [8]:
m1 = torch.rand(5120, 256)
m1s = m1[:4]
m2 = torch.rand(256, 5120)
m2s = m2[:, :4]